In [1]:
import tensorflow as tf
print(tf.__version__)

2024-01-15 17:41:21.733908: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-15 17:41:21.735638: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-15 17:41:21.763114: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-15 17:41:21.763609: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-15 17:41:22.339060: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

2.13.1


In [9]:
#working with MNIST dataset. This dataset contains 70,000 images of handwritten digits
#each image is 28x28 pixels
#each image is labeled with the digit it represents
#the dataset is split into 60,000 images for training and 10,000 images for testing
mnist=tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

#normalize the data
x_train, x_test = x_train/255.0, x_test/255.0 #this is because the pixel values are between 0 and 255 and we want them between 0 and 1


In [10]:
#Building a machine learning model
#the basic building block of a neural network is the layer. Layers extract representations from the data fed into them
#hopefully these representations are more meaningful for the problem at hand
#most of deep learning consists of chaining together simple layers
#most layers, such as tf.keras.layers.Dense, have parameters that are learned during training

#Our model will be a keiras sequential model
#this means that the layers are stacked sequentially
#there are other models such as functional API which is more flexible
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),#the flatten layer transforms the format of the images from a 2d array (of 28 by 28 pixels), to a 1d array (of 28 * 28 = 784 pixels)
  tf.keras.layers.Dense(128, activation='relu'),#this layer has 128 nodes. The dense layer is fully connected to the previous layer and each node contains a score that indicates the current image belongs to one of the 10 classes. The relu activation function is used here which means it will return 0 if it receives any negative input, but for any positive value x it will return that value back
  tf.keras.layers.Dropout(0.2),#the dropout layer helps prevent overfitting by randomly dropping out nodes in the layer during training. This forces the network to learn features in a distributed way and makes the network more robust and able to generalize better to new data. Here 0.2 means 20% of the nodes will be randomly dropped out during training. 
  tf.keras.layers.Dense(10)# This layer returns a logits array with length of 10. Each node contains a score that indicates the current image belongs to one of the 10 classes
])


In [12]:
predictions = model(x_train[:1]).numpy()# predictions is an array of 10 numbers. These describe the "confidence" of the model that the image corresponds to each of the 10 different digits. We can see which label has the highest confidence value using tf.argmax which returns the index of the highest value in a tensor
predictions

array([[ 0.16319257, -0.53564686, -0.11290372,  0.1073992 , -0.12047615,
        -0.0855331 , -0.36822045, -0.7992865 ,  0.96063787,  0.09504695]],
      dtype=float32)

In [13]:
tf.nn.softmax(predictions).numpy()#the softmax function converts these logits to "probabilities" for each class (we can see that all the probabilities add up to 1) and from these probabilities we can see which label has the highest confidence value using tf.argmax which returns the index of the highest value in a tensor (in this case the index of the highest probability) 


array([[0.11289036, 0.05612479, 0.08565442, 0.10676431, 0.08500826,
        0.08803121, 0.06635399, 0.04311789, 0.25060108, 0.10545363]],
      dtype=float32)

In [14]:
# Loss function — This measures how accurate the model is during training. We want to minimize this function to "steer" the model in the right direction.
#Define a loss function for training using losses.SparseCategoricalCrossentropy: This loss function takes a vector of logits and a True index and returns a scalar loss for each example.
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss_fn(y_train[:1], predictions).numpy()#this loss is equal to the negative log probability of the true class: It is zero if the model is sure of the correct class.

#This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to -tf.math.log(1/10) ~= 2.3.

2.4300637

In [15]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [16]:
#Train and evaluate your model. Model.fit method adjusts the model parameters to minimize the loss:
model.fit(x_train, y_train, epochs=5)


Epoch 1/5
1875/1875 [==============================] - 2s 784us/step - loss: 0.2931 - accuracy: 0.9148
Epoch 2/5
1875/1875 [==============================] - 1s 778us/step - loss: 0.1448 - accuracy: 0.9564
Epoch 3/5
1875/1875 [==============================] - 1s 784us/step - loss: 0.1103 - accuracy: 0.9665
Epoch 4/5
1875/1875 [==============================] - 2s 801us/step - loss: 0.0888 - accuracy: 0.9725
Epoch 5/5
1875/1875 [==============================] - 1s 790us/step - loss: 0.0772 - accuracy: 0.9761


In [17]:
#The Model.evaluate method checks the models performance, usually on a "Validation-set" or "Test-set".
model.evaluate(x_test,  y_test, verbose=2)
#The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the TensorFlow tutorials.

313/313 - 0s - loss: 0.0740 - accuracy: 0.9782 - 189ms/epoch - 605us/step


[0.07397665083408356, 0.9782000184059143]

In [18]:
#If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])


In [19]:
probability_model(x_test[:5])


<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[6.0125137e-07, 2.2620577e-08, 2.9771656e-05, 3.4800978e-05,
        4.0169043e-10, 8.7579224e-07, 5.6088315e-12, 9.9992979e-01,
        2.8629154e-07, 3.8603939e-06],
       [1.5043321e-08, 4.8399386e-03, 9.9515116e-01, 7.5881121e-06,
        1.2917784e-13, 7.5782941e-07, 1.3553640e-07, 1.6711700e-12,
        3.7166754e-07, 2.8208917e-12],
       [2.0813570e-06, 9.9917310e-01, 2.8028671e-04, 2.0717993e-05,
        1.9876758e-05, 4.5337106e-06, 5.0645762e-05, 2.6879029e-04,
        1.7879787e-04, 1.0446117e-06],
       [9.9969888e-01, 7.3642788e-08, 9.9956807e-05, 1.8821994e-07,
        7.0509222e-06, 9.8392854e-07, 1.8146733e-04, 7.5803941e-06,
        2.1020442e-08, 3.8913940e-06],
       [5.4087604e-06, 3.9706560e-08, 1.4254672e-05, 1.0667875e-08,
        9.9855381e-01, 7.4322219e-07, 7.9278243e-06, 2.4135634e-05,
        1.5750376e-06, 1.3920533e-03]], dtype=float32)>

In [22]:
x_test[:5].shape

(5, 28, 28)